In [15]:
import math
import cv2
import cvzone
import torch


#import easyocr
import csv
import os
import numpy as np
import pandas as pd
import cv2
import re

import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath


In [3]:
# !pip install cvzon
!pip install easyocr
!pip install paddleocr

  Using cached easyocr-1.7.1-py3-none-any.whl.metadata (11 kB)
  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached scikit_image-0.23.2-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached imageio-2.34.1-py3-none-any.whl.metadata (4.9 kB)
  Using cached lazy_loader-0.4-py3-none-any.whl.metadata (7.6 kB)
Using cached easyocr-1.7.1-py3-none-any.whl (2.9 MB)
Using cached opencv_python_headless-4.9.0.80-cp37-abi3-win_amd64.whl (38.5 MB)
Using cached scikit_image-0.23.2-cp310-cp310-win_amd64.whl (12.7 MB)
Using cached imageio-2.34.1-py3-none-any.whl (313 kB)
Using cached lazy_loader-0.4-py3-none-any.whl (12 kB)
  Using cached paddleocr-2.7.3-py3-none-any.whl.metadata (26 kB)
  Using cached imgaug-0.4.0-py2.py3-none-any.whl.metadata (1.8 kB)
  Using cached visualdl-2.5.3-py3-none-any.whl.metadata (25 kB)
  Using cached opencv_python-4.6.0.66-cp36-abi3-win_amd64.whl.metadata (18 kB)
  Using cached opencv_contrib_python-4.6.0.66-cp36-abi3-

In [16]:
os.chdir("..")

In [17]:
%pwd

'd:\\Deepl learning project\\Object-Detection'

In [124]:
def predict_number_plate(img, ocr):
    result = ocr.ocr(img, cls=True)
    result = result[0]
    texts = [line[1][0] for line in result]
    scores = [line[1][1] for line in result]
    if (scores[0]*100) >= 90:
        return re.sub(r'[^a-zA-Z0-9]', '', texts[0]), scores[0]
    else:
        return None, None

In [28]:
os.system("cd yolov5/ && python detect.py --weights best.pt --img 320 --conf 0.5 --source ../data/inputImage.jpg")

0

In [29]:
result_image = cv2.imread("yolov5/runs/detect/exp/inputImage.jpg")

# Display the resulting image
cv2.imshow("YOLOv5 Detection Result", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [106]:
import os
import cv2

# Execute YOLOv5 detection command
os.system("cd yolov5/ && python detect.py --weights best.pt --img 320 --conf 0.5 --source ../data/inputImage.jpg --save-txt")

# Read the resulting image
result_image = cv2.imread("yolov5/runs/detect/exp/inputImage.jpg")

# Read the output text file containing object coordinates
with open("yolov5/runs/detect/exp/labels/inputImage.txt", "r") as file:
    lines = file.readlines()

# Parse the coordinates and draw bounding boxestrain/images/new5.jpg
class_id = []
for line in lines:
    data = line.strip().split(" ")
    class_id.append(int(data[0]))
    if 1 in class_id:
        
        if 3 in  class_id :
            confidence = float(data[1])
            x_center = float(data[1]) * result_image.shape[1]
            y_center = float(data[2]) * result_image.shape[0]
            width = float(data[3]) * result_image.shape[1]
            height = float(data[4]) * result_image.shape[0]
            
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            
            cv2.rectangle(result_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(result_image, f"Class: {class_id}, Confidence: {confidence:.2f}, "
                                    f"Coords: ({x_center:.2f}, {y_center:.2f})", (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the resulting image
cv2.imshow("YOLOv5 Detection Result", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [34]:
with open("yolov5/runs/detect/exp/labels/inputImage.txt", "r") as file:
    lines = file.readlines()

In [73]:
for line in lines[1:-1]:
    print(line)

3 0.346154 0.64133 0.230769 0.0831354



In [76]:
for line in lines:
    print(line)

2 0.531674 0.545724 0.714932 0.891924

3 0.346154 0.64133 0.230769 0.0831354

1 0.531674 0.186461 0.235294 0.123515



In [97]:
class_id = []
for line in lines:
    data = line.strip().split(" ")
    class_id.append(int(data[0]))
    if 2 == class_id:
        if class_id == 3:
            print("ok")

In [98]:
class_id
    

[2, 3, 1]

In [99]:
class_id == 1

False

In [103]:
if 1 in class_id:
    print("ok")

ok


In [126]:
import os
import cv2

# Execute YOLOv5 detection command
os.system("cd yolov5/ && python detect.py --weights best.pt --img 320 --conf 0.5 --source ../data/inputImage.jpg --save-txt ")

# Read the resulting image
result_image = cv2.imread("yolov5/runs/detect/exp/inputImage.jpg")

# Read the output text file containing object coordinates
with open("yolov5/runs/detect/exp/labels/inputImage.txt", "r") as file:
    lines = file.readlines()

# Parse the coordinates and draw bounding boxestrain/images/new5.jpg
class_id = []
for line in lines:
    data = line.strip().split(" ")
    class_id.append(int(data[0]))
    confidence = float(data[1])
    x_center = float(data[1]) * result_image.shape[1]
    y_center = float(data[2]) * result_image.shape[0]
    width = float(data[3]) * result_image.shape[1]
    height = float(data[4]) * result_image.shape[0]
    if 1 in class_id:
        pass
        if 3 in class_id:
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            cropped_image = result_image[y1:y2, x1:x2]
    # cv2.rectangle(result_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    # cv2.putText(result_image, f"Class: {class_id}, Confidence: {confidence:.2f}, "
    #                         f"Coords: ({x_center:.2f}, {y_center:.2f})", (x1, y1 - 10),
    # cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# Display the resulting image
cv2.imshow("YOLOv5 Detection Result", cropped_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
import easyocr
import uuid
import csv

In [14]:
def save_result(text, region, csv_filename, folder_path):
    image_name = f"{format(uuid.uuid1())}.jpg"
    
    cv2.imwrite(os.path.join(folder_path, image_name), region)
    
    with open(csv_filename, mode='a',newline="") as f:
        Csv_writer = csv.writer(f,delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
        Csv_writer.writerow([image_name,text])

In [15]:
import os
import cv2

# Execute YOLOv5 detection command
os.system("cd yolov5/ && python detect.py --weights best.pt --img 320 --conf 0.5 --source ../data/inputImage.jpg --save-txt")

# Read the resulting image
result_image = cv2.imread("yolov5/runs/detect/exp/inputImage.jpg")

# Read the output text file containing object coordinates
with open("yolov5/runs/detect/exp/labels/inputImage.txt", "r") as file:
    lines = file.readlines()

# Initialize a variable to store the class IDs
class_id = []
reader = easyocr.Reader(['en'])
# Parse the coordinates and find the class IDs
for line in lines:
    data = line.strip().split(" ")
    class_id.append(int(data[0]))

# Check if class 1 and class 3 are present in the detected objects
if 1 in class_id and 3 in class_id:
    for line in lines:
        data = line.strip().split(" ")
        class_id = int(data[0])
        confidence = float(data[1])
        x_center = float(data[1]) * result_image.shape[1]
        y_center = float(data[2]) * result_image.shape[0]
        width = float(data[3]) * result_image.shape[1]
        height = float(data[4]) * result_image.shape[0]
        if class_id == 3:  # Check if the class ID is 3
            x1 = int(x_center - width / 2)
            y1 = int(y_center - height / 2)
            x2 = int(x_center + width / 2)
            y2 = int(y_center + height / 2)
            cropped_image = result_image[y1:y2, x1:x2]
            ocr_result = reader.readtext(cropped_image)
            print(ocr_result)
            save_result(text=ocr_result[0][1],region=cropped_image,csv_filename="number_plate_data.csv",folder_path='run')
            break  # Break out of the loop after finding the first object of class 3

    # Display the cropped image of the first object of class 3
    cv2.imshow("Cropped Image", cropped_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Class 1 or Class 3 not found in the detected objects.")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[([[10, 10], [184, 10], [184, 36], [10, 36]], 'KA 51HG 4702', 0.4884807628487523)]


In [12]:
ocr_result[0][1]

'KA 51HG 4702'

In [157]:
def save_result(text, region, csv_filename, folder_path):
    image_name = f"{format(uuid.uuid1())}.jpg"
    
    cv2.imwrite(os.path.join(folder_path, image_name), region)
    with open(csv_filename, mode='w',newline="") as f:
        Csv_writer = csv.writer(f,delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
        Csv_writer.writerow([image_name,text])

In [155]:
a = ocr_result[1][1]

In [158]:
save_result(text=a,region=cropped_image,csv_filename="number_plate_data.csv",folder_path='run')

In [4]:
%pwd

'd:\\Deepl learning project\\Object-Detection'

In [3]:
os.chdir('..')